<a href="https://colab.research.google.com/github/kungfumas/aplikasi-deep-learning/blob/master/streamlitclasslime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/AmolMavuduru/StreamlitTutorial/master/data/spam_data.csv')
data.head(10)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [3]:
import re # regex library
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # Effectively removes HTML markup tags
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

In [4]:
from sklearn.model_selection import train_test_split
X = data['Message']
y = data['Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

In [6]:
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, max_features=700, preprocessor=preprocessor, ngram_range=(1,1))
neural_net_pipeline = Pipeline([('vectorizer', tfidf), ('nn', MLPClassifier(hidden_layer_sizes=(700, 700)))])
neural_net_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(lowercase=False, max_features=700,
                                 preprocessor=<function preprocessor at 0x7ecc11d95000>)),
                ('nn', MLPClassifier(hidden_layer_sizes=(700, 700)))])

In [7]:
y_pred = neural_net_pipeline.predict(X_test)
print(classification_report(y_test, y_pred))
print('Accuracy: {} %'.format(100 * accuracy_score(y_test, y_pred)))

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1448
        spam       0.95      0.92      0.94       224

    accuracy                           0.98      1672
   macro avg       0.97      0.96      0.96      1672
weighted avg       0.98      0.98      0.98      1672

Accuracy: 98.32535885167464 %


In [12]:
from joblib import dump
dump(neural_net_pipeline, 'spam_classifier.joblib')

['spam_classifier.joblib']

In [14]:
pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=4d678fb38872a04b4b72a7942c2a0c15e3ee3292394b92280e525f05bbc71a28
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [8]:
%%writefile app.py
import joblib
import re
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import streamlit as st
from lime.lime_text import LimeTextExplainer
import streamlit.components.v1 as components

st.write("# Spam Detection Engine")

message_text = st.text_input("Enter a message for spam evaluation")

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # Effectively removes HTML markup tags
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

model = joblib.load('spam_classifier.joblib')

def classify_message(model, message):

	label = model.predict([message])[0]
	spam_prob = model.predict_proba([message])

	return {'label': label, 'spam_probability': spam_prob[0][1]}

if message_text != '':

	result = classify_message(model, message_text)

	st.write(result)


	explain_pred = st.button('Explain Predictions')

	if explain_pred:
		with st.spinner('Generating explanations'):
			class_names = ['ham', 'spam']
			explainer = LimeTextExplainer(class_names=class_names)
			exp = explainer.explain_instance(message_text,
				model.predict_proba, num_features=10)
			components.html(exp.as_html(), height=800)






Writing app.py


In [4]:

!pip install streamlit -qqqq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [5]:
!pip install pyngrok

In [15]:
!streamlit run app.py & npx localtunnel --port 8501

[##................] - fetchMetadata: sill resolveWithNewModule ms@2.1.2 checki

npx: installed 22 in 3.64s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.34.194:8501

your url is: https://rotten-teeth-doubt.loca.lt
  Stopping...
^C


https://blog.streamlit.io/deep-learning-apps-for-image-processing-made-easy-a-step-by-step-guide/

In [8]:
!pip install protobuf

In [9]:
!pip install Pillow

In [11]:
!pip install tensorflow-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 MB 2.6 MB/s eta 0:00:00


In [19]:
! wget https://raw.githubusercontent.com/MainakRepositor/MLDP/master/mango_model.h5

--2023-12-19 07:36:20--  https://raw.githubusercontent.com/MainakRepositor/MLDP/master/mango_model.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53198640 (51M) [application/octet-stream]
Saving to: ‘mango_model.h5’

mango_model.h5      100%[===================>]  50.73M   201MB/s    in 0.3s    

2023-12-19 07:36:21 (201 MB/s) - ‘mango_model.h5’ saved [53198640/53198640]



In [14]:
! wget https://raw.githubusercontent.com/MainakRepositor/MLDP/master/app.py

--2023-12-19 07:28:20--  https://raw.githubusercontent.com/MainakRepositor/MLDP/master/app.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5275 (5.2K) [text/plain]
Saving to: ‘app.py’

app.py              100%[===================>]   5.15K  --.-KB/s    in 0s      

2023-12-19 07:28:20 (58.4 MB/s) - ‘app.py’ saved [5275/5275]



In [17]:
!wget https://raw.githubusercontent.com/MainakRepositor/MLDP/master/mg.png

--2023-12-19 07:31:45--  https://raw.githubusercontent.com/MainakRepositor/MLDP/master/mg.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1010915 (987K) [image/png]
Saving to: ‘mg.png’

mg.png              100%[===================>] 987.22K  --.-KB/s    in 0.05s   

2023-12-19 07:31:45 (20.3 MB/s) - ‘mg.png’ saved [1010915/1010915]



In [7]:
!wget https://github.com/MainakRepositor/MLDP/blob/master/config.toml

--2023-12-19 07:23:12--  https://github.com/MainakRepositor/MLDP/blob/master/config.toml
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4645 (4.5K) [text/plain]
Saving to: ‘config.toml’

config.toml         100%[===================>]   4.54K  --.-KB/s    in 0s      

2023-12-19 07:23:13 (47.7 MB/s) - ‘config.toml’ saved [4645/4645]



In [20]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.196.25.7:8501

npx: installed 22 in 3.039s
your url is: https://fifty-pears-like.loca.lt
2023-12-19 07:36:57.310988: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 07:37:07.073 `st.cache` is deprecated. Please use one of Streamlit's new caching commands,
`st.cache_data` or `st.cache_resource`.

More information [in our docs](https://docs.streamlit.io/library/advanced-features/caching).
2023-12-19 07:37:07.075 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. P